In [54]:
import os
import sys
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from constants import openai_key
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA


os.environ["OPENAI_API_KEY"]=openai_key

# sys.path.append('../..')

# from dotenv import load_dotenv, find_dotenv
# _ = load_dotenv(find_dotenv()) # read local .env file

# openai.api_key  = os.environ['OPENAI_API_KEY']


In [2]:
loader = PyPDFLoader("Attention_Is_All_You_Need.pdf")
pages = loader.load()

In [3]:
len(pages)

15

In [4]:
page = pages[0]

In [5]:
print(page.page_content[0:500])

Attention Is All You Need
Ashish Vaswani∗
Google Brain
avaswani@google.comNoam Shazeer∗
Google Brain
noam@google.comNiki Parmar∗
Google Research
nikip@google.comJakob Uszkoreit∗
Google Research
usz@google.com
Llion Jones∗
Google Research
llion@google.comAidan N. Gomez∗†
University of Toronto
aidan@cs.toronto.eduŁukasz Kaiser∗
Google Brain
lukaszkaiser@google.com
Illia Polosukhin∗‡
illia.polosukhin@gmail.com
Abstract
The dominant sequence transduction models are based on complex recurrent or
conv


In [6]:
page.metadata

{'source': 'Attention_Is_All_You_Need.pdf', 'page': 0}

In [ ]:
chunk_size = 26
chunk_overlap = 4

In [ ]:
r_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap
)
c_splitter = CharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap
)
r_splitter1 = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
    length_function=len,
    is_separator_regex=False,
)
c_splitter1 = CharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,length_function=len,
    is_separator_regex=False,
)
r_splitter2 = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
    length_function=len,
    is_separator_regex=True,
)
c_splitter2 = CharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,length_function=len,
    is_separator_regex=True,
)

In [ ]:

text1 = "abcdefghijklomnpqrstuvwxyz"
text2 = "abcdefghijklomnpqrstuvwxyzabcdefgh"

In [ ]:
output1 = r_splitter.split_text(text1)
output2 = r_splitter.split_text(text2)
print(f"Output1:{output1} and Output2:{output2}")

In [ ]:
output1 = r_splitter.split_text(text1)
output11 = r_splitter1.split_text(text1)
output111 = r_splitter2.split_text(text1)

print(f"Output1:{output1} and Output11:{output1} Output111:{output111}")
output1 = r_splitter.split_text(text2)
output11 = r_splitter1.split_text(text2)
output111 = r_splitter2.split_text(text2)

print(f"Output1:{output1} and Output11:{output1} Output111:{output111}")

In [ ]:
text = """When writing documents, writers will use document structure to group content. \
This can convey to the reader, which idea's are related. For example, closely related ideas \
are in sentances. Similar ideas are in paragraphs. Paragraphs form a document. \n\n  \
Paragraphs are often delimited with a carriage return or two carriage returns. \
Carriage returns are the "backslash n" you see embedded in this string. \
Sentences have a period at the end, but also, have a space.\
and words are separated by space."""

In [ ]:
len(text)

In [19]:
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=1000,
    chunk_overlap=150,
    length_function=len
)
r_splitters = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=150, 
    separators="\n",
    length_function=len
)

In [20]:
docs = text_splitter.split_documents(pages)
docs1 = r_splitters.split_documents(pages)


In [22]:
print(f"length of Documents after Character Text Splitter: {len(docs)} and length of Orginal Documents before Character Text Splitter: {len(pages)} with chunk_size=1000, chunk_overlap=150")
print(f"length of Documents after Recursive Character Text Splitter: {len(docs1)} and length of Orginal Documents before Recursive Character Text Splitter: {len(pages)} with chunk_size=1000, chunk_overlap=150")


length of Documents after Character Text Splitter: 49 and length of Orginal Documents before Character Text Splitter: 15 with chunk_size=1000, chunk_overlap=150
length of Documents after Recursive Character Text Splitter: 49 and length of Orginal Documents before Recursive Character Text Splitter: 15 with chunk_size=1000, chunk_overlap=150


In [8]:
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=1500,
    chunk_overlap=150,
    length_function=len
)
r_splitters = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=150, 
    separators="\n",
    length_function=len
)

In [30]:
docs = text_splitter.split_documents(pages)
docs1 = r_splitters.split_documents(pages)


In [31]:
print(f"length of Documents after Character Text Splitter: {len(docs)} and length of Orginal Documents before Character Text Splitter: {len(pages)} with chunk_size=850, chunk_overlap=200")
print(f"length of Documents after Recursive Character Text Splitter: {len(docs1)} and length of Orginal Documents before Recursive Character Text Splitter: {len(pages)} with chunk_size=850, chunk_overlap=200")


length of Documents after Character Text Splitter: 62 and length of Orginal Documents before Character Text Splitter: 15 with chunk_size=850, chunk_overlap=200
length of Documents after Recursive Character Text Splitter: 62 and length of Orginal Documents before Recursive Character Text Splitter: 15 with chunk_size=850, chunk_overlap=200


In [2]:

# Load PDF
loaders = [
    # Duplicate documents on purpose - messy data
    PyPDFLoader("Attention_Is_All_You_Need.pdf"),
    PyPDFLoader("Attention_Is_All_You_Need.pdf"),
    PyPDFLoader("BERT.pdf"),
    PyPDFLoader("SCIBERT.pdf"),
    PyPDFLoader("FinGPT.pdf"),
    PyPDFLoader("FinBERT.pdf")
]
docs = []
for loader in loaders:
    docs.extend(loader.load())


r_splitters = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=150, 
    separators="\n",
    length_function=len
)

splits = r_splitters.split_documents(docs)

In [3]:
len(splits)

222

In [4]:
embedding = OpenAIEmbeddings()

persist_directory = 'docs/chroma/'

!rm -rf ./docs/chroma  # remove old database files if any

'rm' is not recognized as an internal or external command,
operable program or batch file.


In [5]:
vectordb = Chroma.from_documents(
    documents=splits,
    embedding=embedding,
    persist_directory=persist_directory
)
print(vectordb._collection.count())

444


In [6]:
question = "what is BERT?"

In [7]:
docs = vectordb.similarity_search(question,k=5)

In [8]:
len(docs)

5

In [9]:
docs[0].page_content

'BERT: Pre-training of Deep Bidirectional Transformers for\nLanguage Understanding\nJacob Devlin Ming-Wei Chang Kenton Lee Kristina Toutanova\nGoogle AI Language\n{jacobdevlin,mingweichang,kentonl,kristout }@google.com\nAbstract\nWe introduce a new language representa-\ntion model called BERT , which stands for\nBidirectional Encoder Representations from\nTransformers. Unlike recent language repre-\nsentation models (Peters et al., 2018a; Rad-\nford et al., 2018), BERT is designed to pre-\ntrain deep bidirectional representations from\nunlabeled text by jointly conditioning on both\nleft and right context in all layers. As a re-\nsult, the pre-trained BERT model can be ﬁne-\ntuned with just one additional output layer\nto create state-of-the-art models for a wide\nrange of tasks, such as question answering and\nlanguage inference, without substantial task-\nspeciﬁc architecture modiﬁcations.\nBERT is conceptually simple and empirically\npowerful. It obtains new state-of-the-art re-\nsu

In [10]:
docs[1].page_content

'BERT: Pre-training of Deep Bidirectional Transformers for\nLanguage Understanding\nJacob Devlin Ming-Wei Chang Kenton Lee Kristina Toutanova\nGoogle AI Language\n{jacobdevlin,mingweichang,kentonl,kristout }@google.com\nAbstract\nWe introduce a new language representa-\ntion model called BERT , which stands for\nBidirectional Encoder Representations from\nTransformers. Unlike recent language repre-\nsentation models (Peters et al., 2018a; Rad-\nford et al., 2018), BERT is designed to pre-\ntrain deep bidirectional representations from\nunlabeled text by jointly conditioning on both\nleft and right context in all layers. As a re-\nsult, the pre-trained BERT model can be ﬁne-\ntuned with just one additional output layer\nto create state-of-the-art models for a wide\nrange of tasks, such as question answering and\nlanguage inference, without substantial task-\nspeciﬁc architecture modiﬁcations.\nBERT is conceptually simple and empirically\npowerful. It obtains new state-of-the-art re-\nsu

In [11]:
vectordb.persist()

In [12]:
for doc in docs:
    print(doc.metadata)

{'page': 0, 'source': 'BERT.pdf'}
{'page': 0, 'source': 'BERT.pdf'}
{'page': 0, 'source': 'SCIBERT.pdf'}
{'page': 0, 'source': 'SCIBERT.pdf'}
{'page': 3, 'source': 'FinBERT.pdf'}


### Addressing Diversity: Maximum marginal relevance


In [13]:
docs_mmr = vectordb.max_marginal_relevance_search(question,k=2)

In [14]:
for doc_mmr in docs_mmr:
    print(doc_mmr.metadata)

{'page': 0, 'source': 'BERT.pdf'}
{'page': 2, 'source': 'SCIBERT.pdf'}


In [15]:
docs_mmr[0].page_content[:100]

'BERT: Pre-training of Deep Bidirectional Transformers for\nLanguage Understanding\nJacob Devlin Ming-W'

In [16]:
docs_mmr[1].page_content[:100]

'size of 32 and a learning rate of 0.001.\nWe did not perform extensive hyperparameter\nsearch, but whi'

### Addressing Specificity: working with metadata


In [28]:
questions = "On which machine they trained the model"

In [29]:
docs = vectordb.similarity_search(
    questions,
    k=3,
    filter={"source":"Attention_Is_All_You_Need.pdf"}
)

In [30]:
for d in docs:
    print(d.metadata)

{'page': 6, 'source': 'Attention_Is_All_You_Need.pdf'}
{'page': 6, 'source': 'Attention_Is_All_You_Need.pdf'}
{'page': 6, 'source': 'Attention_Is_All_You_Need.pdf'}


### Addressing Specificity: working with metadata using self-query retriever


In [31]:
metadata_field_info = [
    AttributeInfo(
        name="source",
        description="The lecture the chunk is from, should be one of `Attention_Is_All_You_Need.pdf`, `BERT.pdf`, or `SCIBERT.pdf`",
        type="string",
    ),
    AttributeInfo(
        name="page",
        description="The page from the lecture",
        type="integer",
    ),
]

In [32]:
document_content_description = "Lecture notes"
llm = OpenAI(model='gpt-3.5-turbo-instruct', temperature=0)
retriever = SelfQueryRetriever.from_llm(
    llm,
    vectordb,
    document_content_description,
    metadata_field_info,
    verbose=True
)

In [33]:
docs_1 = retriever.get_relevant_documents(question)
docs_2 = retriever.get_relevant_documents(questions)

In [34]:
print(f"Question 1 answer: {docs_1} \n Question 2 answer: {docs_2}")

Question 1 answer: [Document(page_content='BERT \nE[CLS] E1 E[SEP] ... ENE1’... EM’\nC\nT1\nT[SEP] ...\n TN\nT1’...\n TM’\n[CLS] Tok \n1 [SEP] ...Tok \nNTok \n1...Tok\nM\nQuestion Paragraph BERT \nE[CLS] E1 E2 EN\nC\nT1\n T2\n TN\nSingle Sentence ...\n...BERT \nTok 1 Tok 2 Tok N ... [CLS]E[CLS] E1 E2 EN\nC\nT1\n T2\n TN\nSingle Sentence \nB-PER O O...\n... E[CLS] E1 E[SEP] Class \nLabel \n... ENE1’... EM’\nC\nT1\nT[SEP] ...\n TN\nT1’...\n TM’\nStart/End Span Class \nLabel \nBERT \nTok 1 Tok 2 Tok N ... [CLS]Tok 1[CLS] [CLS] Tok \n1 [SEP] ...Tok \nNTok \n1...Tok\nM\nSentence 1 \n...Sentence 2 \nFigure 4: Illustrations of Fine-tuning BERT on Different Tasks.\nSST-2 The Stanford Sentiment Treebank is a\nbinary single-sentence classiﬁcation task consist-\ning of sentences extracted from movie reviews\nwith human annotations of their sentiment (Socher\net al., 2013).\nCoLA The Corpus of Linguistic Acceptability is\na binary single-sentence classiﬁcation task, where\nthe goal is to predict w

In [35]:
for d in docs_1:
    print(d.metadata)

{'page': 14, 'source': 'BERT.pdf'}
{'page': 14, 'source': 'BERT.pdf'}
{'page': 0, 'source': 'BERT.pdf'}
{'page': 0, 'source': 'BERT.pdf'}


In [36]:
for d in docs_2:
    print(d.metadata)

{'page': 10, 'source': 'Attention_Is_All_You_Need.pdf'}
{'page': 10, 'source': 'Attention_Is_All_You_Need.pdf'}
{'page': 10, 'source': 'Attention_Is_All_You_Need.pdf'}
{'page': 10, 'source': 'Attention_Is_All_You_Need.pdf'}


# compression approach

In [39]:
def pretty_print_docs(docs):
    print(f"\n{'-' * 100}\n".join([f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]))


In [40]:
# Wrap our vectorstore
llm = OpenAI(temperature=0)
compressor = LLMChainExtractor.from_llm(llm)

In [41]:
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=vectordb.as_retriever()
)

In [43]:
question = "what did they say about postional?"
compressed_docs = compression_retriever.get_relevant_documents(question)
pretty_print_docs(compressed_docs)

c:\Users\Aai\anaconda3\envs\pytorch\Lib\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
c:\Users\Aai\anaconda3\envs\pytorch\Lib\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
c:\Users\Aai\anaconda3\envs\pytorch\Lib\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
c:\Users\Aai\anaconda3\envs\pytorch\Lib\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


Document 1:

McClosky et al. (2006) [26] semi-supervised 92.1
Vinyals & Kaiser el al. (2014) [37] semi-supervised 92.1
Transformer (4 layers) semi-supervised 92.7
Luong et al. (2015) [23] multi-task 93.0
Dyer et al. (2016) [8] generative 93.3
In Table 3 rows (B), we observe that reducing the attention key size dkhurts model quality.
sinusoidal positional encoding with learned positional embeddings [ 9], and observe nearly identical
results to the base model.
----------------------------------------------------------------------------------------------------
Document 2:

McClosky et al. (2006) [26] semi-supervised 92.1
Vinyals & Kaiser el al. (2014) [37] semi-supervised 92.1
Transformer (4 layers) semi-supervised 92.7
Luong et al. (2015) [23] multi-task 93.0
Dyer et al. (2016) [8] generative 93.3
In Table 3 rows (B), we observe that reducing the attention key size dkhurts model quality.
This suggests that determining compatibility is not easy and that a more sophisticated compatibility


In [44]:
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=vectordb.as_retriever(search_type = "mmr")
)

In [45]:
question = "what did they say about d_model?"
compressed_docs = compression_retriever.get_relevant_documents(question)
pretty_print_docs(compressed_docs)

c:\Users\Aai\anaconda3\envs\pytorch\Lib\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
c:\Users\Aai\anaconda3\envs\pytorch\Lib\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
c:\Users\Aai\anaconda3\envs\pytorch\Lib\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
c:\Users\Aai\anaconda3\envs\pytorch\Lib\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


Document 1:

In Table 3 rows (B), we observe that reducing the attention key size d_model hurts model quality.
----------------------------------------------------------------------------------------------------
Document 2:

dmodel = 1024, beam size of 21 and α= 0.3


In [46]:
question = "What are major topics for this class?"
docs = vectordb.similarity_search(question,k=3)
len(docs)

3

In [47]:
import datetime
current_date = datetime.datetime.now().date()
if current_date < datetime.date(2023, 9, 2):
    llm_name = "gpt-3.5-turbo-0301"
else:
    llm_name = "gpt-3.5-turbo"
print(llm_name)

gpt-3.5-turbo


In [50]:
llm = ChatOpenAI(model_name=llm_name, temperature=0)
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever()
)

In [51]:
result = qa_chain({"query": question})

In [52]:
result["result"]

'Based on the provided context, the major topics for this class are:\n\n1. Financial polarity analysis: This involves classifying text as positive, negative, or neutral in the context of financial domains.\n\n2. Pre-trained language models: These are models that have been trained on large amounts of data and can be used for various natural language processing tasks.\n\n3. Automated sentiment analysis: This refers to the use of natural language processing methods to analyze and derive insights from financial texts produced by financial actors.\n\n4. Challenges in polarity analysis: The challenges include the need for large amounts of labeled data, the specialized language and vocabulary used in financial texts, and the tendency to use vague expressions instead of easily-identified negative/positive words.\n\n5. Financial sentiment lexicons: These are lexicons that incorporate existing financial knowledge into textual analysis, but they are based on "word counting" methods.\n\nPlease not

In [55]:

# Build prompt
template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible. Always say "thanks for asking!" at the end of the answer. 
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

In [56]:
# Run chain
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

In [57]:
question = "what are the different hyperparameters were used?"

In [58]:
result = qa_chain({"query":question})

In [59]:
result["result"]

'The different hyperparameters used were the number of attention heads and the attention key and value dimensions. Thanks for asking!'

In [63]:
question = "which machine they used to train the model?"
result = qa_chain({"query":question})
result["result"]

'They trained their models on one machine with 8 NVIDIA P100 GPUs. Thanks for asking!'

In [70]:
question = "which datasets were used to train the models?"
result = qa_chain({"query":question})
result["result"]

'The datasets used to train the models are TRC2-financial and Financial PhraseBank. Thanks for asking!'

In [71]:
question = "what is Position-wise Feed-Forward Networks?"
result = qa_chain({"query":question})
result["result"]

'Position-wise Feed-Forward Networks are fully connected feed-forward networks that are applied to each position separately and identically in both the encoder and decoder layers. They consist of two linear transformations with a ReLU activation in between. The dimensionality of the input and output is 512, and the inner-layer has a dimensionality of 2048. Thanks for asking!'

In [60]:
qa_chain_mr = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    chain_type="map_reduce"
)

In [61]:
result = qa_chain({"query":question})

In [62]:
result['result']

'The different hyperparameters used were the number of attention heads and the attention key and value dimensions. Thanks for asking!'